Check this article first if you speak Vietnamese, it outlines the steps implemented.

https://viblo.asia/p/b5-hopskipjumpattack-a-query-efficient-decision-based-attack-L4x5xLGm5BM

In [2]:
# imports
import numpy as np
from typing import Callable, Optional

Binary search to find the boundary.

In [3]:
def binsearch_boundary(src_pt: np.ndarray,
                      dest_pt: np.ndarray,
                      threshold: float,
                      fn: Callable[[np.ndarray], bool]
                     ) -> np.array:
    '''
    Find a point between two points that will lies on the boundary.
    :param src_pt:    point at which phi=0
    :param dest_pt:   point at which phi=1
    :param threshold: gap between source point and destination point
    :param fn:        function that takes in a point and returns T/F if phi=1/0.
    '''
    while np.linalg.norm(dest_pt - src_pt) >= threshold:
        midpoint = (src_pt + dest_pt) / 2
        if fn(midpoint):
            dest_pt = midpoint
        else:
            src_pt = midpoint
    return dest_pt

Estimate gradient when on the boundary.

In [96]:
def estimate_gradient(orig_pt: np.ndarray,
                      step_size: np.ndarray,
                      sample_count: int,
                      fn: Callable[[np.ndarray], bool]
                     ) -> np.ndarray:
    '''
    Estimate the gradient via Monte Carlo sampling.
    :param orig_pt:      point to estimate gradient at
    :param step_size:    length of each step in the proposed direction
    :param sample_count: number of Monte Carlo samples
    :param fn:           function that takes in a point and returns T/F if phi=1/0.
    '''
    # sample directions
    directions = np.random.randn(orig_pt.shape[0], sample_count)
    directions /= np.linalg.norm(directions, axis=0)
    # get phi values
    values = np.empty((orig_pt.shape[0], 1), dtype=np.float)
    for i in range(sample_count):
        values[i, 0] = fn(directions[:, i]) * 2 - 1
    # subtract from the mean
    values -= np.mean(values)
    # and average them
    avg = np.sum(directions * values, axis=1) / (sample_count - 1)
    # project them to unit L2
    return avg / np.linalg.norm(avg)

Gradient descent with estimated gradient.

In [97]:
def gradient_descent(orig_pt: np.ndarray,
                     grad: np.ndarray,
                     step_size: float,
                     fn: Callable[[np.ndarray], bool]
                    ) -> np.ndarray:
    '''
    Do gradient descent on a point already on the boundary.
    :param orig_pt:    point to do gradient descent on
    :param grad:       the estimated gradient
    :param step_size:  initial step size to try
    :param fn:         function that takes in a point and returns T/F if phi=1/0.
    '''
    # find the step size to stay in phi=1
    while True:
        new_vector = orig_pt + step_size * grad
        if fn(new_vector):
            break
        step_size /= 2
    return new_vector

And the whole HopSkipJumpAttack in all its glory.

In [102]:
def hopskipjumpattack(orig_pt: np.ndarray,
                      fn: Callable[[np.ndarray], bool],
                      max_iter: Optional[int] = 100,
                      init_grad_queries: Optional[int] = 100,
                      binsearch_threshold: Optional[float] = 1e-10,
                      dest_pt: Optional[np.ndarray] = None,
                      start_iter: Optional[int] = 0
                     ) -> np.ndarray:
    '''
    Implementation of the HopSkipJumpAttack.
    :param orig_pt:             point at which phi=0
    :param fn:                  function that takes in a point and returns T/F if phi=1/0
    :param max_iter:            (Optional) maximum number of optimization iteration.
                                Default is 100.
    :param init_grad_queries:    (Optional) initial query count to estimate gradient
                                Default is 100.
    :param binsearch_threshold: (Optional) the threshold to stop binary searching the boundary.
                                Default is 1e-6.
    :param dest_pt:             (Optional) point which phi=1.
                                If dest_pt is None, will be initialized to be a random vector.
    :param start_iter:          (Optional) last iteration count.
                                For cases when one restarts this iterative algo. Default is 0.
    '''
    d = orig_pt.shape[0]
    # initialize a vector with phi=1
    if dest_pt is None:
        while True:
            dest_pt = np.random.random_sample(d)
            if fn(dest_pt):
                break
    for it in range(start_iter + 1, max_iter + 1):
        print(f'Iter {it:03d}: ', end='')
        # find the boundary
        boundary = binsearch_boundary(orig_pt, dest_pt, binsearch_threshold, fn)
        # if the error is too small, return as is
        distance = np.linalg.norm(boundary - orig_pt)
        if distance < binsearch_threshold:
            print(distance)
            print('Too small step size, terminated.')
            # this works because we return the phi=1 endpoint in binsearch.
            return boundary
        # estimate the gradient
        step_size = np.linalg.norm(dest_pt - orig_pt) / d
        sample_count = int(init_grad_queries * it ** 0.5)
        grad = estimate_gradient(boundary, step_size, sample_count, fn)
        # and gradient descend
        step_size = np.linalg.norm(boundary - orig_pt) / it ** 0.5
        dest_pt = gradient_descent(boundary, grad, step_size, fn)
        distance = np.linalg.norm(dest_pt - orig_pt)
        print(distance)
    return dest_pt

Now the rest is just calling it. Again, `fn` refers to a function that returns True if the datapoint is successfully misclassified (the specifics depends on whether it's targeted or untargeted), and False otherwise.

*Note: this repo is for learning purposes, there is already implementations by the author (@Jianbo-Lab) and on various adversarial toolboxes (CleverHans/Foolbox).*